In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint

print('Versão do TensorFlow: ', tf.__version__)


### Import of CSV


In [ ]:
df = pd.read_csv('./data.csv',skiprows=0) 
input_data = df.iloc[:, 0:4].values
out_data = df.iloc[:, 4:] .values


### Mapping classes to replace it for the values


In [ ]:
# Mapeamento de classes para valores numéricos
classe_para_valor = {
    "CACTUS_SMALL": 0,
    "CACTUS_LARGE": 1,
    "PTERODACTYL": 2
}
 
acao_para_valor = {
    "JUMP": 0,
    "CROUCHED": 1,
    "RUN": 2,
}


### Replacing String for Values


In [ ]:
# Converter dados para valores numéricos
input_data = np.array([[classe_para_valor[classe], dist_x, dist_y, dist_z] for classe, dist_x, dist_y , dist_z in input_data])
out_data = np.array([acao_para_valor[acao[0]] for acao in out_data])  # Ajuste aqui


### Creating the model


In [ ]:

# Crie o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(3, activation='softmax')  # num_classes é o número de ações possíveis
])

### Compiling the model


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy( ),
              metrics=['accuracy'])

### Starting the trainning


In [ ]:
%load_ext tensorboard
log_dir = "./logs/fit/"
weights = "./logs/fit/weights/weights.h5"
csv_log_dir = log_dir+'/trainning.log'
backup = './logs/fit/backup'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
 
model.fit(input_data, out_data, epochs=100, verbose=1,
                callbacks=[
                tensorboard_callback,
                tf.keras.callbacks.CSVLogger(csv_log_dir),
                tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.001),
                tf.keras.callbacks.ModelCheckpoint(filepath= weights, save_best_only=True),
                tf.keras.callbacks.BackupAndRestore(
                backup_dir= backup,
                save_freq  = "epoch",
                delete_checkpoint = True,
                save_before_preemption  = False
           )]
          );

### Evaluating the model


In [ ]:

# Avalie o modelo
loss, accuracy = model.evaluate(input_data, out_data)
print("Accuracy:", accuracy)

### Saving the model


In [ ]:
model.save("./dino_model",save_format='tf')

### Exporting to tensorflowJS


In [ ]:
! tensorflowjs_converter   --input_format=tf_saved_model  --metadata= ./dino_model ../src/assets/web_dino_model